# Question 2:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

w = np.arange(-5., 5., 0.02)
u1 = w**4
u2 = np.log10(w[251:])
u3 = np.power(2,w)
u4 = np.tanh(w)

plt.plot(w, u1, 'c--')
plt.grid(True)
plt.title('w^4')
plt.show()

plt.plot(w[251:], u2, 'c--')
plt.grid(True)
plt.title('log(w)')
plt.show()

plt.plot(w, u3, 'c--')
plt.grid(True)
plt.title('2^w')
plt.show()

plt.plot(w, u4, 'c--')
plt.grid(True)
plt.title('tanh(w)')
plt.show()



# Question 4:

Hypothesis 1:

In [98]:
import numpy as np

def findGuardStartCell(cells):
    cellIndex = np.random.choice(np.arange(0,3))
    return cellIndex

def findNextCell(cellsProb):
    return np.random.choice(np.arange(0,3), p=cellsProb)
        
def updateCellProb(cells, cellsProb, currCell, numIter=2, cellsChoiceNum = []):
    
    if all(i==0 for i in cellsProb) or numIter < 5:
        for i in range(3):
            if not i == currCell:
                cellsProb[i] = 0.5
        cellsProb[currCell] = 0  
        
    elif numIter > 4 :
        sumAllChoices = sum(cellsChoiceNum)
        for i in range(3):
            cellsProb[i] = cellsChoiceNum[i] / sumAllChoices
    
    return cellsProb

def updateGuardPath(guardPathInfo, currCell, nextCell, cells):
    allMoves = list(guardPathInfo.keys())
    
    for i in range(6):
        if allMoves[i][0] == cells[currCell] and allMoves[i][1] == cells[nextCell]:
            guardPathInfo[allMoves[i]] = guardPathInfo[allMoves[i]] + 1
            
    return guardPathInfo

def updatePathInfo(guardPathInfo):
    allMoves = list(guardPathInfo.keys())
    
    for i in range(6):
        if i%2 == 0:
            totalCellMoves = guardPathInfo[allMoves[i]] + guardPathInfo[allMoves[i+1]]
            guardPathInfo[allMoves[i]] = guardPathInfo[allMoves[i]] / totalCellMoves
            guardPathInfo[allMoves[i+1]] = guardPathInfo[allMoves[i+1]] / totalCellMoves
    return guardPathInfo        
    
def findProb(cells, startCell, cellsChoiceNum, guardPathInfo):
    #setting initial probability
    #iteration 1
    cellsProb = [0 for i in range(3)]
    cellsProb = updateCellProb(cells, cellsProb, startCell)
    nextCell = findNextCell(cellsProb)
    cellsChoiceNum[nextCell] = 1
    guardPathInfo = updateGuardPath(guardPathInfo, startCell, nextCell, cells)
    
    for i in range(2,1000):
        #iteration > 1
        currCell = nextCell
        cellsProb = updateCellProb(cells, cellsProb, currCell, i, cellsChoiceNum)
        nextCell = findNextCell(cellsProb)
        cellsChoiceNum[nextCell] = cellsChoiceNum[nextCell] + 1 
        guardPathInfo = updateGuardPath(guardPathInfo, currCell, nextCell, cells)
        
    return updatePathInfo(guardPathInfo)


cells = ['a', 'b', 'c']
cellsChoiceNum = [0, 0, 0]
guardPathInfo = {'ab':0, 'ac':0, 'ba':0, 'bc':0, 'ca':0, 'cb':0}

guardStartCell = findGuardStartCell(cells)
cellsProb = findProb(cells, guardStartCell, cellsChoiceNum, guardPathInfo)
print(cellsProb)


{'ab': 0.1510791366906475, 'ac': 0.8489208633093526, 'ba': 0.1935483870967742, 'bc': 0.8064516129032258, 'ca': 0.6269430051813472, 'cb': 0.37305699481865284}


Hypothesis 2:

In [101]:
import numpy as np


def findGuardStartCell(cells):
    cellIndex = np.random.choice(np.arange(0,3))
    return cellIndex

def findNextCell(cellsProb):
    return np.random.choice(np.arange(0,3), p=cellsProb)

def findPathProbGuard(guardPathProb, currCell):
    allMoves = list(guardPathProb.keys())
    nextCellInfo = []
    
    for i in range(6):
        if allMoves[i][0] == cells[currCell]:
            nextCellInfo.append(allMoves[i])
            nextCellInfo.append(guardPathProb[allMoves[i]])
    
    return nextCellInfo

def updateCellProb(cells, cellsProb, currCell, guardPathProb):
    
    cellsProb[currCell] = 0
    nextCellInfo = findPathProbGuard(guardPathProb, currCell)
    
    for i in range(4):
        if i%2 == 0:
            for j in range(3):
                if cells[j] == nextCellInfo[i][1]:
                    cellsProb[j] = nextCellInfo[i+1]
    return cellsProb
    

def updateGuardPath(guardPathProbGuess, currCell, nextCell, cells):
    allMoves = list(guardPathProbGuess.keys())
    
    for i in range(6):
        if allMoves[i][0] == cells[currCell] and allMoves[i][1] == cells[nextCell]:
            guardPathProbGuess[allMoves[i]] = guardPathProbGuess[allMoves[i]] + 1
            
    return guardPathProbGuess

def findPathProbGuess(guardPathProbGuess):
    allMoves = list(guardPathProbGuess.keys())
    
    for i in range(6):
        if i%2 == 0:
            totalCellMoves = guardPathProbGuess[allMoves[i]] + guardPathProbGuess[allMoves[i+1]]
            guardPathProbGuess[allMoves[i]] = guardPathProbGuess[allMoves[i]] / totalCellMoves
            guardPathProbGuess[allMoves[i+1]] = guardPathProbGuess[allMoves[i+1]] / totalCellMoves
    return guardPathProbGuess        
    
def aBeliefUpdating(cells, startCell, guardPathProbGuess, guardPathInfo):
    #setting initial probability
    #iteration 1
    cellsProb = [0 for i in range(3)]
    cellsProb = updateCellProb(cells, cellsProb, startCell, guardPathProb)
    nextCell = findNextCell(cellsProb)
    guardPathProbGuess = updateGuardPath(guardPathProbGuess, startCell, nextCell, cells)
    
    for i in range(2,10000):
        #iteration > 1
        currCell = nextCell
        cellsProb = updateCellProb(cells, cellsProb, currCell, guardPathProb)
        nextCell = findNextCell(cellsProb)
        guardPathProbGuess = updateGuardPath(guardPathProbGuess, currCell, nextCell, cells)
           
    return findPathProbGuess(guardPathProbGuess)


cells = ['a', 'b', 'c']
cellsChoiceNum = [0, 0, 0]
guardPathProb = {'ab':0.1, 'ac':0.9, 'ba':0.8, 'bc':0.2, 'ca':0.3, 'cb':0.7}
guardPathProbGuess = {'ab':0, 'ac':0, 'ba':0, 'bc':0, 'ca':0, 'cb':0}

guardStartCell = findGuardStartCell(cells)
guardPathProbGuess = aBeliefUpdating(cells, guardStartCell, guardPathProbGuess, guardPathInfo)
print(guardPathProbGuess)


{'ab': 0.0960352422907489, 'ac': 0.9039647577092511, 'ba': 0.7958904109589041, 'bc': 0.20410958904109588, 'ca': 0.2942297223734349, 'cb': 0.7057702776265651}
